In [1]:
# import vars 
import integration.lightsheet_evap as lib
import os
import numpy as np

In [2]:
# Variable names for M-Loop optimization
var_names = [
    'i_hf_lightsheet_evap1_current',
    'v_pd_hf_lightsheet_rampdown_end', 
    't_hf_lightsheet_rampdown',
    'v_pd_hf_lightsheet_rampdown2_end',
    't_hf_lightsheet_rampdown2',
    'i_hf_lightsheet_evap2_current'
]

# Current values (you can modify these as needed)
var_space = [
    [185.0, 195.],   # i_hf_lightsheet_evap1_current
    [0.23, 4.],    # v_pd_hf_lightsheet_rampdown_end
    [0.05, 2.5],    # t_hf_lightsheet_rampdown
    [0.2, 0.3],    # v_pd_hf_lightsheet_rampdown2_end
    [0.5, 4.],     # t_hf_lightsheet_rampdown2
    [185., 195.]    # i_hf_lightsheet_evap2_current
]



In [3]:
#make table for vars
import pandas as pd


from kexp.config import expt_params

# Create an instance of the class
example_instance = expt_params.ExptParams()

# Prepare data for the table
data = []
for i, name in enumerate(var_names):
    default_value = getattr(example_instance, name, None)
    min_range, max_range = var_space[i]
    data.append([name, default_value, min_range, max_range])

# Create a DataFrame
df = pd.DataFrame(data, columns=["Variable Name", "Default Value", "Minimum Range Value", "Maximum Range Value"])

# Print the DataFrame
print(df)



                      Variable Name  Default Value  Minimum Range Value  \
0     i_hf_lightsheet_evap1_current         189.00               185.00   
1   v_pd_hf_lightsheet_rampdown_end           1.00                 0.23   
2          t_hf_lightsheet_rampdown           0.64                 0.05   
3  v_pd_hf_lightsheet_rampdown2_end           0.25                 0.20   
4         t_hf_lightsheet_rampdown2           0.74                 0.50   
5     i_hf_lightsheet_evap2_current         193.30               185.00   

   Maximum Range Value  
0                195.0  
1                  4.0  
2                  2.5  
3                  0.3  
4                  4.0  
5                195.0  


In [4]:
def main():
    #M-LOOP can be run with three commands
    
    #First create your interface
    interface = lib.CustomInterface(var_names)
    #Next create the controller. 
    #The controller must take the variables to be changed in the vars object
    #vars = Param()


    controller = lib.mlc.create_controller(interface, 
                                       controller_type = 'neural_net',  #type of controller to use, can be 'neural_network', 'gaussian process', 'differential evolution', 'random', 'nelder_mead''
                                       max_num_runs = 1000,
                                       target_cost = -109090000000,
                                       num_params = len(var_names),
                                       first_params = [189.4,1,0.8,0.26,1.9,193.3], 
                                       min_boundary = np.transpose(var_space)[0],
                                       max_boundary = np.transpose(var_space)[1],
                                       interface_file_type = 'txt',               #file types of *exp_input.mat* and *exp_output.mat*
                                       controller_archive_file_type = 'txt',      #file type of the controller archive
                                       learner_archive_file_type = 'txt')      #file type of the learner archive
    controller.optimize()

main()

INFO     M-LOOP version 3.3.4
INFO     Optimization started.
INFO     Run: 0 (training)
INFO     params [189.4    1.     0.8    0.26   1.9  193.3 ]
0 Run id: 35532
 1 values of dummy. 1 total shots. 3 total images expected.
basler
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 35532
Parameters saved, data closed.
Done!
 
No ROI saved in run 35532.
ROI specified by Run ID. Attempting to load ROI...
ROI loaded from run 35531.
Using ROI loaded from run 35531.
-19989.209448797556
INFO     cost -19989.209448797556 +/- 0.0
INFO     Run: 1 (training)
INFO     params [185.57729275   0.28628014   1.60875792   0.24147281   3.14941287
 186.09168616]
0 Run id: 35533
 1 values of dummy. 1 total shots. 3 total images expected.
basler
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 35533
Parameters saved, data closed.
Done!
 
No ROI saved in run 35533.
ROI specified by Run ID. Attempting to load ROI...
ROI loaded from run 35531.
Using ROI loaded from run 35531.
-10383.7751161

In [ ]:
def param_format(param_values,param_keys=var_names):
    lines = ""
    for i in range(len(param_keys)):
            lines += f"""
                self.p.{param_keys[i]} = {float(param_values[i])}"""
    print(lines)

param_format([-3.15221506,  0.10830731, -1.20206951, -2.93364217,  0.17977458,  0.1727242,
  1.96822209, -1.59931595, -3.91874333,  0.2,         0.09954919, 17.40341653])


                self.p.detune_push = -3.15221506
                self.p.amp_push = 0.10830731
                self.p.detune_d2_c_2dmot = -1.20206951
                self.p.detune_d2_r_2dmot = -2.93364217
                self.p.amp_d2_c_2dmot = 0.17977458
                self.p.amp_d2_r_2dmot = 0.1727242
                self.p.v_2d_mot_current = 1.96822209
                self.p.detune_d2_c_mot = -1.59931595
                self.p.detune_d2_r_mot = -3.91874333
                self.p.amp_d2_c_mot = 0.2
                self.p.amp_d2_r_mot = 0.09954919
                self.p.i_mot = 17.40341653


In [ ]:
def print_default_params(param_keys=var_names):
    from kexp.config import ExptParams
    p = ExptParams()
    lines = ""
    for key in param_keys:
            lines += f"""
                self.p.{key} = {float(vars(p)[key])}"""
    print(lines)
print_default_params()


                self.p.detune_push = -2.4
                self.p.amp_push = 0.13
                self.p.detune_d2_c_2dmot = -1.2
                self.p.detune_d2_r_2dmot = -2.4
                self.p.amp_d2_c_2dmot = 0.188
                self.p.amp_d2_r_2dmot = 0.188
                self.p.v_2d_mot_current = 2.11
                self.p.detune_d2_c_mot = -2.4
                self.p.detune_d2_r_mot = -4.2
                self.p.amp_d2_c_mot = 0.188
                self.p.amp_d2_r_mot = 0.188
                self.p.i_mot = 20.0


In [ ]:
def print_mloop_param_bounds(param_keys=var_names,param_bounds=var_space):
    lines = ""
    for i in range(len(param_keys)):
            lines += f"""
                self.p.{param_keys[i]} \u2208 {var_space[i]}"""
    print(lines)
print_mloop_param_bounds()


                self.p.detune_push ∈ [-5.4, 0.6]
                self.p.amp_push ∈ [0.01, 0.2]
                self.p.detune_d2_c_2dmot ∈ [-4.2, 1.8]
                self.p.detune_d2_r_2dmot ∈ [-5.4, 0.6]
                self.p.amp_d2_c_2dmot ∈ [0.01, 0.2]
                self.p.amp_d2_r_2dmot ∈ [0.01, 0.2]
                self.p.v_2d_mot_current ∈ [0.5, 5.0]
                self.p.detune_d2_c_mot ∈ [-5.4, 0.6]
                self.p.detune_d2_r_mot ∈ [-7.2, -1.2]
                self.p.amp_d2_c_mot ∈ [0.01, 0.2]
                self.p.amp_d2_r_mot ∈ [0.01, 0.2]
                self.p.i_mot ∈ [15.0, 25.0]


In [ ]:
def print_param_report(mloop_params,param_keys=var_names,param_bounds=var_space):
    from kexp.config import ExptParams
    p = ExptParams()
    lines = ""
    for i in range(len(param_keys)):
            line = []
            line.append(f"self.p.{param_keys[i]}".ljust(25,' '))
            line.append(f" \u2208 {var_space[i]}:".ljust(20,' '))
            line.append(f"mloop = {mloop_params[i]:1.3f},   ".ljust(18,' '))
            line.append(f"default = {vars(p)[param_keys[i]]:1.3f}".ljust(15,' '))
            s = ""
            for L in line:
                s += L
            lines += f"""
                {s}"""
    return lines
lines = print_param_report([-3.15221506,  0.10830731, -1.20206951, -2.93364217,  0.17977458,  0.1727242,
  1.96822209, -1.59931595, -3.91874333,  0.2,         0.09954919, 17.40341653])
print(lines)


                self.p.detune_push        ∈ [-5.4, 0.6]:     mloop = -3.152,   default = -2.400
                self.p.amp_push           ∈ [0.01, 0.2]:     mloop = 0.108,    default = 0.130
                self.p.detune_d2_c_2dmot  ∈ [-4.2, 1.8]:     mloop = -1.202,   default = -1.200
                self.p.detune_d2_r_2dmot  ∈ [-5.4, 0.6]:     mloop = -2.934,   default = -2.400
                self.p.amp_d2_c_2dmot     ∈ [0.01, 0.2]:     mloop = 0.180,    default = 0.188
                self.p.amp_d2_r_2dmot     ∈ [0.01, 0.2]:     mloop = 0.173,    default = 0.188
                self.p.v_2d_mot_current   ∈ [0.5, 5.0]:      mloop = 1.968,    default = 2.110
                self.p.detune_d2_c_mot    ∈ [-5.4, 0.6]:     mloop = -1.599,   default = -2.400
                self.p.detune_d2_r_mot    ∈ [-7.2, -1.2]:    mloop = -3.919,   default = -4.200
                self.p.amp_d2_c_mot       ∈ [0.01, 0.2]:     mloop = 0.200,    default = 0.188
                self.p.amp_d2_r_mot       ∈ 

In [ ]:
#### uncomment this and run it if u wanna delete the logs


# import os, shutil
# # Specify the path of the file to be deleted
# folder = r'C:\Users\bananas\code\k-exp\kexp\experiments\Mloop testing\M-LOOP_archives'
# file_path_2 = r'C:\Users\bananas\code\k-exp\kexp\experiments\Mloop testing\M-LOOP_logs'

# for filename in os.listdir(folder):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete %s. Reason: %s' % (file_path, e))

# for filename in os.listdir(file_path_2):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete %s. Reason: %s' % (file_path, e))